In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import NMT

In [2]:
!pip install datasets

In [3]:
from NMT import train_translate, models, BCN, Data_prep_WMT16

2025-01-29 21:11:09.864147: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-29 21:11:09.882020: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-29 21:11:09.900536: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-29 21:11:09.906019: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-29 21:11:09.921596: I tensorflow/core/platform/cpu_feature_guar

In [4]:
from NMT.train_translate import *

In [5]:
#

GLOVE_PATH = "./data/glove.6B.300d.txt"
EMBS_PATH = "./data/prep/glove.embs.pickle"
VOCAB_PATH = "./data/prep/vocab.json"
TRAIN_SRC_PATH = "./data/prep/train.tok.en"
TRAIN_TGT_PATH = "./data/prep/train.tok.de"
VAL_SRC_PATH = "./data/prep/val.tok.en"
VAL_TGT_PATH = "./data/prep/val.tok.de"
TEST_SRC_PATH = "./data/prep/test.tok.en"
TEST_TGT_PATH = "./data/prep/test.tok.de"
MODEL_SAVE_PATH = "./checkpoints/"
TOKENIZER_EN_PATH = "./data/prep/tokenizer.en.pickle"
TOKENIZER_DE_PATH = "./data/prep/tokenizer.de.pickle"

In [6]:
TRAIN_SRC_PATH

'./data/prep/train.tok.en'

In [7]:
from NMT.Data_prep_WMT16 import preprocess_wmt

In [8]:
#Preprocess the downloaded data
train_en, train_de, val_en, val_de, test_en, test_de, embedding_matrix, vocab, i2w, tokenizer_en, tokenizer_de = preprocess_wmt("./data/", "./data/glove.6B.300d.txt", "./data/prep/")
#get loaders from data
dataloader = create_dataloader(train_en, train_de, 32)
val_dl = create_dataloader(val_en, val_de, 32)
en_vocab = tokenizer_en.word_index
de_vocab = tokenizer_de.word_index

 Gloves saved to ./data/prep/glove.embs.pickle
 Tokenizer saved to ./data/prep/tokenizer.en.pickle
 Tokenizer saved to ./data/prep/tokenizer.de.pickle


In [9]:
dataloader = create_dataloader(train_en, train_de, 32)
val_dl = create_dataloader(val_en, val_de, 32)
en_vocab = tokenizer_en.word_index
de_vocab = tokenizer_de.word_index
embedding_tensor = torch.from_numpy(embedding_matrix).float().to(device)

In [10]:
#load the preprocessed data
import pickle
with open("./data/prep/tokenizer.en.pickle", "rb") as f:
        tokenizer_en = pickle.load(f)
with open("./data/prep/tokenizer.de.pickle", "rb") as f:
        tokenizer_de = pickle.load(f)
with open(EMBS_PATH, "rb") as f:
        combined_embeddings = pickle.load(f)
embedding_tensor = torch.from_numpy(combined_embeddings).float().to(device)
src_sentences, tgt_sentences = load_tokenized_data(TRAIN_SRC_PATH, TRAIN_TGT_PATH)
dataloader = create_dataloader(src_sentences, tgt_sentences, 32)
val_src_sentences, val_tgt_sentences = load_tokenized_data(VAL_SRC_PATH, VAL_TGT_PATH)
val_dl = create_dataloader(val_src_sentences, val_tgt_sentences, 32)
en_vocab = tokenizer_en.word_index
de_vocab = tokenizer_de.word_index

In [10]:
print(f"en vocab {len(en_vocab)} embeddings {embedding_tensor.shape} de vocab {len(de_vocab)}")

en vocab 18855 embeddings torch.Size([50000, 300]) de vocab 43315


In [11]:
model = initialize_model(en_vocab, de_vocab, embedding_tensor)

In [10]:
train_model(model, dataloader, validation_dl= val_dl, vocab_size=len(de_vocab), epochs = 100)

Epoch [1/100] - Loss: 4.7584
Validation Loss: 14.8474
Checkpoint saved at ./checkpoints/best_checkpoint.pth
Best model updated with Validation Loss: 14.8474
Epoch [2/100] - Loss: 4.5146
Validation Loss: 14.5802
Epoch [3/100] - Loss: 4.0544
Validation Loss: 13.8854
Epoch [4/100] - Loss: 3.7236
Validation Loss: 13.2893
Epoch [5/100] - Loss: 3.3745
Validation Loss: 12.9168
Epoch [6/100] - Loss: 3.1454
Validation Loss: 12.1383
Epoch [7/100] - Loss: 2.9046
Validation Loss: 11.8349
Epoch [8/100] - Loss: 2.7592
Validation Loss: 11.1501
Epoch [9/100] - Loss: 2.4608
Validation Loss: 10.44
Epoch [10/100] - Loss: 2.0866
Validation Loss: 10.2392
Epoch [11/100] - Loss: 1.9794
Validation Loss: 9.9003
Epoch [12/100] - Loss: 1.7968
Validation Loss: 9.5641
Epoch [13/100] - Loss: 1.426
Validation Loss: 8.8381
Epoch [14/100] - Loss: 1.0541
Validation Loss: 8.3593
Epoch [15/100] - Loss: 0.7617
Validation Loss: 7.8915
Epoch [16/100] - Loss: 0.5675
Validation Loss: 7.5884
Epoch [17/100] - Loss: 0.2926
Valid

In [13]:
model.eval()

NMTModel(
  (encoder): Encoder(
    (dropout_l): Dropout(p=0.2, inplace=False)
    (dropout_l2): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(50000, 300, padding_idx=0)
    (LSTM): LSTM(300, 150, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(43315, 300, padding_idx=0)
    (dropout_l1): Dropout(p=0.2, inplace=False)
    (LSTM): LSTM(600, 300, num_layers=2, batch_first=True, dropout=0.2)
    (dropout_l2): Dropout(p=0.2, inplace=False)
    (linear_attn_in): Linear(in_features=300, out_features=300, bias=True)
    (linear_attn_out): Linear(in_features=600, out_features=300, bias=True)
    (dropout_l3): Dropout(p=0.2, inplace=False)
  )
  (linear): Linear(in_features=300, out_features=43315, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [14]:
test_src_sentences, test_tgt_sentences = load_tokenized_data(TEST_SRC_PATH, TEST_TGT_PATH)

In [15]:
test_dl = create_dataloader(test_src_sentences, test_tgt_sentences, 32)

In [26]:
next(iter(test_dl))[0]

tensor([[   3, 4329,  169,  ...,    0,    0,    0],
        [   3,  184, 4067,  ...,    0,    0,    0],
        [   3,    2, 1700,  ...,    0,    0,    0],
        ...,
        [   3,   33,    2,  ...,    0,    0,    0],
        [   3, 6934,  150,  ...,    0,    0,    0],
        [   3,   13,  178,  ...,    0,    0,    0]])

In [20]:
tokenizer_de

In [5]:
tokenizer_en.sequences_to_texts(next(iter(dataloader))[1])

0	1 7656 29494 2592 618 12544 9503 15708 330 680 13206 1 5091 12741 2475 1 857 2170 104 3853 12335 10058 2553 2994 1832 162 7222 1 104 1 1 4581 1 5327 21582 23897 104 21603 45593 13206
0	1422 23440 37202 1076 283 4304 1685 12208 18119 21204 3612 1 48826 35598 1 1 1 23082 1596 12142 1 7723 6539 1 5600 20962 1 12820 2575 10939 6850 1502 2456 6995 5435 40686 30243 7039 1 23082
0	26041 8461 26041 8461 26041 8461 257 9820 197 1979 7836 2647 1 618 1 1221 6610 1685 389 23940 6850 1 37339 31010 2182 1 1 26584 585 10939 8344 1 127 565 1 13831 13831 6573 2398 1136
0	1 32093 14138 1 1 1 494 13844 6950 19757 20934 1 4117 4869 1 1401 8574 6849 1 21965 21965 3784 40491 4869 1 43 16031 2747 585 1 1 8020 1531 17912 2134 2981 31059 1 1 19496
0	10472 13841 23447 2981 198 62 19795 1010 2160 330 661 10744 2189 7146 4137 7176 4869 11761 1 1 8020 7841 240 1 1305 1 27945 19028 1396 1832 1880 31103 1252 7592 1 62 8020 24481 1 100
0	3726 2747 33244 6640 1 1 1834 1 1 40491 11900 24007 1 40491 10520 1 293 25495 